# Módulo de preprocesamiento 
Ammi Beltrán & Fernanda Borja

## 1

In [62]:
# ! pip install -U scikit-learn

     ---------------------------------------- 9.2/9.2 MB 25.7 MB/s eta 0:00:00
     ---------------------------------------- 302.2/302.2 kB ? eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\TheSy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [64]:
import os
import mne
import glob
#
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

import copy

In [27]:
if not os.path.exists("data"):
    os.makedirs("data")
    print("Data directory created :D")

In [3]:
# EDFDIR = "D:\\OneDrive\\OneDrive - Universidad de Chile\\Semestre X\\Inteligencia\\Proyecto\\dataset\\tuh_eeg"

In [28]:
EDFDIR = "c:\\Users\\TheSy\\Desktop\\tuh_eeg"

In [50]:
files = glob.glob(EDFDIR + '/**/*.edf', recursive=True)

In [51]:
# Usamos MNE
data = mne.io.read_raw_edf(files[0])
raw_data = data.get_data()
info = data.info
channels = data.ch_names

Extracting EDF parameters from c:\Users\TheSy\Desktop\tuh_eeg\aaaaaaaa\s001_2015_12_30\01_tcp_ar\aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [1]:
#CANALES
'''
CHANNELS = ['EEG FP1-REF','EEG FP2-REF',
            'EEG F7-REF','EEG F8-REF','EEG F4-REF','EEG F3-REF','EEG FZ-REF',
            'EEG A1-REF','EEG A2-REF','EEG O1-REF','EEG O2-REF',
            'EEG T6-REF','EEG T5-REF','EEG T4-REF','EEG T3-REF',
            'EEG C4-REF','EEG C3-REF','EEG CZ-REF',
            'EEG P4-REF','EEG P3-REF','EEG PZ-REF',
            ]
CHANNELS_LE = [
            'EEG FP1-LE',
            'EEG FP2-LE',
            'EEG F7-LE',
            'EEG F8-LE',
            'EEG F4-LE',
            'EEG F3-LE',
            'EEG FZ-LE',
            'EEG A1-LE',
            'EEG A2-LE',
            'EEG O1-LE',
            'EEG O2-LE',
            'EEG T6-LE',
            'EEG T5-LE',
            'EEG T4-LE',
            'EEG T3-LE',
            'EEG C4-LE',
            'EEG C3-LE',
            'EEG CZ-LE',
            'EEG P4-LE',
            'EEG P3-LE',
            'EEG PZ-LE',
            ]
'''
CHANNELS = [
            ' FP1',
            ' FP2',
            ' F7',
            ' F8',
            ' F4',
            ' F3',
            ' FZ',
            ' A1',
            ' A2',
            ' O1',
            ' O2',
            ' T6',
            ' T5',
            ' T4',
            ' T3',
            ' C4',
            ' C3',
            ' CZ',
            ' P4',
            ' P3',
            ' PZ',
            ]

In [ ]:
# patient = glob.glob(EDFDIR + '/**')
# patient

In [69]:

def preprocessing(path,):
    ''' 
    Pipeline de procesamiento c:
    - Canales c:
    - Filtrado c:
    - Downsample c:
    - Crop temporal (primer minuto y a los 11 minutos) c:  
    - Segmentación c:
    - Normalización c:
    '''

    files = glob.glob(path + '/**/*.edf', recursive = True)
    # windows = np.empty() EVALUAR
    windows = []
    for i in range(0, len(files)):
        data = mne.io.read_raw_edf(files[i], preload=True)
        # raw_data = data.get_data()
        # info = data.info
        channels = data.ch_names[:21]
        # Pipeline
        # 
        # mapping = rename_channels(data)

        # data.rename_channels(data.info, mapping)
        # data.rename_channels(mapping)

        # try:
        channel_data = channel_select(data, channels)
        clip(channel_data,channels)
        # except:
        #     channel_data = channel_select(data, CHANNELS_LE)
        #     ch = CHANNELS_LE
        if(int(data.times[-1]) < 100):
            continue
        filtered = eeg_filter(channel_data)
        trimmed_data = temporal_crop(filtered)
        epochs = get_epochs(trimmed_data, channels)
        down_data = downsample(epochs)
        norm_data = normalization(down_data)
        #
        windows.append(norm_data)
    return windows


In [67]:
# Channels we want

def rename_channels(data):
    # info = data.info
    original = data.ch_names
    new = copy.copy(original)
    res = {}
    for ch in new:
        
        isin = False
        for n in CHANNELS:
            if n in ch:
                res[ch] = n
                isin = True
                break

        if not isin:
            res[ch] = ch
            
    return res


# def rename_channels(data):
#     # info = data.info
#     original = data.ch_names
#     print(original)
#     new = copy.copy(original)
#     awa = [original[i] if (new[i] not in original[i]) else new[i] for i in range(0, len(new))]
#     res = {original[i]: awa[i] for i in range(len(awa))}
#     return res
    

In [70]:
win = preprocessing(EDFDIR)
win

Extracting EDF parameters from c:\Users\TheSy\Desktop\tuh_eeg\aaaaaaaa\s001_2015_12_30\01_tcp_ar\aaaaaaaa_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 323839  =      0.000 ...  1264.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 80 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.30, 80.00 Hz: -6.02, -6.02 dB

Not setting metadata
33 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 33 events and 5121 original time points ...
0 bad epochs dropped
Dropped 1 epoch: 32
Extracting EDF parameters from c:\Users\TheSy\Desktop\tuh_eeg\aaaaaaab\s001_2002_12_30\02_tcp_le\aaaaaaab_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info stru

IndexError: Epoch index -1 is out of bounds

In [56]:
def channel_select(data, channels):
    '''
    Selects channels from array 
    '''
    extracted = data.pick(channels, exclude="bads")
    return extracted

In [57]:
def eeg_filter(data, lfreq = 0.3, hfreq= 80):
    '''
    
    '''
    data_copy = copy.copy(data)
    filtered = data_copy.filter(l_freq = lfreq,
                                h_freq = hfreq,
                                method = "iir"
                                )
    return filtered

In [58]:
def temporal_crop(data, tin = 60, tfin = 12*60):
    ''' 
    Cut the channels from the second "tin" to "tfin"
    '''
    data_copy = copy.copy(data)
    croped = data_copy.crop(tmin = tin, tmax = min(tfin, int(data.times[-1])))
    return croped

In [59]:
def downsample(epoch, freq = 200):
    ''' 
    Downsamples the data given by a factor
    En nuestro caso, down corresponde a (frecuencia que queremos)/(frecuencia actual)
    '''
    down = epoch.resample(freq, npad = "auto")
    return down

In [60]:
# eventos
def get_epochs(data, channels, window = 20.0):
    ''' 
    window es la ventana de tiempo
    '''
    data_copy = copy.copy(data)
    # Create events
    events = mne.make_fixed_length_events(data_copy, duration = window, first_samp = True)
    # Divide accordingly
    picks = channels
    epochs = mne.Epochs(raw = data_copy, events = events, picks = picks, preload = True,
                        tmin = 0., tmax = window, baseline = None,
                        flat = dict(eeg = 1e-6))
    
    epochs.drop(-1,reason = "Unfixed duration")
    return epochs


In [68]:
def clip(data, channels,max= 500e-6):
    def cliper(array):
        for i in range(len(array)):
            if abs(array[i]) > max:
                array[i] = math.copysign(max,array[i])
        return array
    data.apply_function(cliper, picks=channels, channel_wise= True)

In [61]:
def normalization(epochs):
    obj = mne.decoding.Scaler(info = epochs.info, scalings='mean')
    values = obj.fit_transform(epochs.get_data())
    return values